In [16]:
# packages
import os 
import sys
import random
import numpy as np 
from numpy import linalg as LA
from time import process_time

In [17]:
# method to read data from file
# OUTPUT: adjacency matrix
def read(filename):
    PROJECT_PATH = "./DIMACS/"
    path_to_file = os.path.join(PROJECT_PATH, filename)
    
    with open(path_to_file, 'r') as file:
        for line in file:
            if line.startswith('c'):  # graph description
                line.split()[1:]
                #print(*line.split()[1:])
            # first line: p name num_of_vertices num_of_edges
            elif line.startswith('p'):
                p, name, vertices_num, edges_num = line.split()
                print('{0} {1} {2}\n'.format(name, vertices_num, edges_num))
                adj = np.zeros((int(vertices_num),int(vertices_num)))
            elif line.startswith('e'):
                _, v1, v2 = line.split()
                adj[int(v1)-1][int(v2)-1]=1
                adj[int(v2)-1][int(v1)-1]=1
            else:
                continue
                
        #return adjacency matrix        
        return int(vertices_num), adj  

In [18]:
num, adj=read("c500.9.clq") #Take a look at the graph size/ adjacency

col 500 112332



COMPLEMENTARY METHODS:

In [19]:
# output the function value of 0.5*x'Ax
def func_val(x, Q):
    val = 0.5*x.transpose().dot(Q).dot(x)
    return val

# check the final result if it is indeed a clique
def check(adj,index):
    for i in index:
        for j in index:
            if i!=j and adj[i][j]!=1:
                return "false"
            
    return "true"

MAIN ALGORITHM:

In [20]:
#Armijo Backtracking
def case_2(vertice_num, x, adj, max_iters, delta, mu, epsilon):
    # Modify the adjacency matrix
    Id = np.identity(vertice_num)
    Q= -(adj+1/2*Id)
    L = np.sqrt(np.trace(Q.dot(Q)))     # L=root(trace(A'A))
    
    for iteration in range(max_iters):
        
##1     # Hessian derived from entropy function
        hessian = np.diag(1/(x**2))

    
##2     # step direction
        Qx = Q.dot(x) ; x_sqr = x**2
        v_x = -x_sqr*(Qx - x_sqr.dot(Qx)/sum(x_sqr))
    
##3     # step-size
        #--descent inequality 
        vHv = (v_x/x_sqr).dot(v_x)        
        alpha = 0.95*(2*vHv)/(L*v_x.dot(v_x))

        
##4     # set test point        
        x_hat = x + alpha*v_x

        if x_hat.min()<0:
            np.min([x[i]/(-v_x[i]) for i in range(vertice_num) if v_x[i]<0]) # alpha_0= min(x_i/(-v_i))            


##5     # sufficient decrease
        f_val = func_val(x, Q)
        f_add = mu*vHv
        while func_val(x_hat, Q) > f_val - alpha*f_add:
            alpha = alpha*delta     #shrink step size
            x_hat = x + alpha*v_x     # update test point

##6     # stopping Rule
        if (-func_val(x_hat,Q)+f_val)<(epsilon):     
            break

##7     # new state
        old_x = x   
        x = x_hat  


    return x

INITIALIZE X, RUN ALGORITHM THEN OUTPUT FINAL RESULT

In [21]:
def solve(filename):
    # read the adjacency matrix 
    vertice_num, adj = read(filename)
    print("SOLUTION: ")
    
    # choose starting point at the middle of the simplex
    x=np.array([1/vertice_num for i in range(vertice_num)]) 
    
    # RUN ALGORITHM 
    # max 10^6 iteration, shrinking factor delta=0.5, mu=0.001, epsilon=10^(-9) chosen
    start = process_time()
    sol = case_2(vertice_num, x, adj, 1000000, 0.5, 0.01, 0.000000001)
    end = process_time()
    
    # retrieve the index of chosen vertices
    ans = np.array([i for i in sol if i>0.1/len(sol)])
    index = np.array([i for i in range(len(sol)) if sol[i]>0.1/len(sol)])
    
    print(' '.join(str(x+1) for x in index))
    print("Maximum clique size: {}\n".format(len(ans)))
    print("Clique: {}; sum to {}".format(check(adj,index),sum(ans)))
    print('CPU = {}'.format(np.mean(end-start)))
    
    return sol

SOLVE HERE

In [71]:
solution = solve("keller4.clq")  # epsilon = 10^(-9)

edge 171 9435

SOLUTION: 
0.3283082621118062
0.3318072781140593
0.3349603783367614
0.33794071848246066
0.34080928634503205
0.34354162409876843
0.346052838714336
0.34826686670383983
0.35017924702172837
0.3518406615210189
0.3533118293070468
0.35464254431150166
0.3558692988234764
0.35701819200031387
0.3581080361650191
0.35915265243570804
0.36016242286069633
0.36114532078837625
0.3621075992487722
0.36305425810744085
0.36398936709082685
0.3649162934874658
0.36583786566118826
0.3667564925319265
0.36767425229735073
0.3685929592933122
0.3695142150635376
0.3704394478503461
0.37136994347550256
0.3723068697360189
0.3732512958570309
0.37420420813603067
0.37516652262317574
0.3761390954743424
0.377122731462316
0.37811819102028693
0.37912619610931797
0.38014743513974014
0.38118256712985676
0.3822322252501824
0.38329701987469733
0.3843775412403966
0.3854743618012324
0.38658803835148914
0.3877191139857572
0.388868119957835
0.3900355774984785
0.391221999652121
0.39242789319543014
0.3936537607060479
0.39

0.4642078118372603
0.46420781576658393
0.46420781964217855
0.46420782346478395
0.46420782723512954
0.4642078309539348
0.46420783462190884
0.46420783823975215
0.4642078418081547
0.46420784532779635
0.4642078487993492
0.46420785222347505
0.4642078556008263
0.4642078589320478
0.4642078622177739
0.4642078654586314
0.4642078686552378
0.46420787180820205
0.46420787491812554
0.46420787798560026
0.4642078810112111
0.46420788399553436
0.46420788693913817
0.46420788984258354
0.46420789270642293
0.4642078955312019
0.4642078983174581
0.46420790106572263
0.46420790377651727
0.46420790645035925
0.4642079090877561
0.46420791168921033
0.464207914255217
0.4642079167862636
0.46420791928283195
0.4642079217453965
0.4642079241744258
0.46420792657038135
0.4642079289337187
0.4642079312648868
0.464207933564329
0.46420793583248127
0.46420793806977545
0.4642079402766361
0.46420794245348207
0.4642079446007271
0.4642079467187785
0.4642079488080386
0.4642079508689034
0.4642079529017641
0.4642079549070065
0.4642079

In [7]:
# RUN WITH MULTIPLE STARTING POINTS 

def multi_solve(filename, trials):
    # input the adjacency matrix
    vertice_num, adj = read(filename)

    
    #running trials of different starting x
    print('{} trials:'.format(trials)) 
    solutions = []; time = []
    Id = np.identity(vertice_num)
    A= -(adj+1/2*Id)
    
    for i in range(trials):
        print("process: {}".format(i+1))
        # choose the starting point at random
        x = np.random.dirichlet(np.ones(vertice_num)).ravel()
 
        # run algorithm
        start = process_time()
        sol = case_2(vertice_num, x, adj, 1000000, 0.5, 0.01, 0.000000001)
        end = process_time()
    
        # retrieve the index of the chosen vertices
        ans = np.array([i for i in sol if i>0.1/len(sol)]) 
        index = np.array([i for i in range(len(sol)) if sol[i]>0.1/len(sol)])
        
        if check(adj,index):
            solutions.append(len(ans))
            time.append(end-start)
    
    return np.array(solutions), np.array(time)

In [11]:
file = "c2000.5.clq"

In [12]:
solutions,times = multi_solve(file, 100)

col 2000 999836

100 trials:
process: 1
process: 2
process: 3
process: 4
process: 5
process: 6
process: 7
process: 8
process: 9
process: 10
process: 11
process: 12
process: 13
process: 14
process: 15
process: 16
process: 17
process: 18
process: 19
process: 20
process: 21
process: 22
process: 23
process: 24
process: 25
process: 26
process: 27
process: 28
process: 29
process: 30
process: 31
process: 32
process: 33
process: 34
process: 35
process: 36
process: 37
process: 38
process: 39
process: 40
process: 41
process: 42
process: 43
process: 44
process: 45
process: 46
process: 47
process: 48
process: 49
process: 50
process: 51
process: 52
process: 53
process: 54
process: 55
process: 56
process: 57
process: 58
process: 59
process: 60
process: 61
process: 62
process: 63
process: 64
process: 65
process: 66
process: 67
process: 68
process: 69
process: 70
process: 71
process: 72
process: 73
process: 74
process: 75
process: 76
process: 77
process: 78
process: 79
process: 80
process: 81
process:

In [13]:
print('{} return true clique'.format(len(solutions)))
print('max = {}'.format(np.amax(solutions)))
print('mean = {}'.format(np.mean(solutions)))
print('std = {}'.format(np.std(solutions)))
print('CPU = {}'.format(np.mean(times)))

100 return true clique
max = 13
mean = 11.02
std = 0.824378553820998
CPU = 304.32765625


In [14]:
result.append([file,len(solutions),np.amax(solutions),np.mean(solutions),np.std(solutions),np.mean(times)])

In [15]:
result

[['c125.9.clq', 100, 33, 28.67, 1.8550202155232702, 2.670625],
 ['c250.9.clq', 100, 39, 34.85, 1.86748493969831, 11.575],
 ['c500.9.clq', 100, 47, 42.23, 2.0969263220246916, 38.88125],
 ['p_hat300-1.clq', 100, 7, 5.92, 0.7166589146867568, 20.24640625],
 ['c1000.9.clq', 100, 53, 49.07, 1.6140322177701407, 60.49984375],
 ['DSJC500_5.clq',
  75,
  12,
  9.226666666666667,
  0.8258060035841078,
  110.47729166666667],
 ['brock200_4.clq', 75, 14, 11.76, 1.1412274094149684, 11.291458333333333],
 ['brock400_2.clq',
  75,
  22,
  17.973333333333333,
  1.1997036671148793,
  38.97833333333333],
 ['brock400_4.clq', 50, 22, 18.04, 1.326046756340062, 64.705],
 ['gen200_p0.9_44.clq', 50, 35, 31.76, 1.8927229062913569, 5.5115625],
 ['gen200_p0.9_55.clq', 50, 37, 34.12, 1.6203703280423276, 2.3440625],
 ['gen200_p0.9_55.clq', 50, 37, 34.12, 1.6203703280423276, 2.3440625],
 ['c2000.5.clq', 100, 13, 11.02, 0.824378553820998, 304.32765625]]

In [62]:
times

array([ 20.46875 ,  40.859375,  20.65625 ,  20.125   ,  39.390625,
        41.171875,  23.5     ,  21.859375,  17.25    ,  30.5     ,
        46.28125 ,  41.328125,  33.25    , 575.171875,  14.90625 ,
        25.109375,  22.109375,  22.921875,  16.65625 ,  40.171875,
         9.875   ,   9.515625,  16.875   , 321.796875,  12.140625,
        12.234375,  14.859375,   8.390625,  12.515625,   6.734375,
        19.84375 , 430.421875,  26.890625,  11.015625,   6.734375,
         9.609375,  10.203125,   7.34375 ,  13.875   ,  16.671875,
        12.359375,  16.96875 ,  18.640625,  19.890625,  29.59375 ,
        24.453125,  35.53125 ,  13.625   ,  12.5     ,  23.78125 ,
        18.5     ,  17.453125,  16.890625,  17.515625,  14.375   ,
        25.5625  ,  23.203125,  17.09375 ,  11.703125,  13.359375,
        23.8125  ,  11.90625 ,  16.359375,  14.6875  ,   9.5625  ,
       474.859375,  14.640625,   9.15625 ,   9.265625,  12.625   ,
        17.3125  ,  10.1875  ,  14.0625  ,  15.015625,  15.25 